In [1]:
from init import *

In [2]:
import math
import numpy as np

class Example1UKF (UnscentedKalmanFilter):
    '''A rotating disk with some marker.
    State:
    - disk radius
    - initial angle of the marker
    - rotation speed
    
    Measurement:
    - x coordinate of the marker
    - y ...
    '''
    
    def __init__ (self):
        super().__init__(3,2)

        # the true state (used in this test)
        self.__true_state = np.matrix([
            [11],
            [22],
            [0.01]
        ], dtype=np.float64)
        
        # initial state estimate
        self.x = np.matrix([
            [0],
            [0],
            [0]
        ], dtype=np.float64)

        # initial state error matrix
        big = 10
        self.P = np.matrix([
            [big, 0 , 0 ],
            [ 0, big, 0 ],
            [ 0,  0 ,big]
        ], dtype=np.float64)
        
        self.monitor = KalmanFilterMonitor(self)
        
    def GetIterationTime (self,iteration):
        return float(iteration)
        
    def GetPredictedMeasurement (self, state, iteration):
        '''override'''
        assert state.shape == (3,1)
        
        radius = state[0,0]
        angle0 = state[1,0]
        rpm    = state[2,0]
        time   = self.GetIterationTime (iteration)
        phi    = angle0+time*rpm*2*math.pi

        return np.matrix([
            [radius*math.cos(phi)],
            [radius*math.sin(phi)]
        ], dtype=np.float64)
        
    def GetMeasurement (self, iteration):
        '''override'''
        xy_error = 0.5
        error = np.matrix([
            [np.random.default_rng().normal(0,xy_error)],
            [np.random.default_rng().normal(0,xy_error)]
        ], dtype=np.float64)
        
        error_matrix = np.matrix ([
            [xy_error**2,  0],
            [0,  xy_error**2],
        ], dtype=np.float64)
        measurement_value = self.GetPredictedMeasurement(self.__true_state, iteration) + error
        return (measurement_value, error_matrix)
    
    def Log (self, iteration, what, value):
        if not hasattr(value, '__iter__'):
            return self.Log(what,[value])
        
        if 0:
            delimiter = '-'*30
            print(f'Iteration={iteration} {what}')
            for p in value:
                print(p)
            print(delimiter)
        
        self.monitor.AddData(iteration, what, value)
        
    def GetTrueState (self):
        return self.__true_state
    
    def GetResidualLength (self, residual):
        v = 0 
        for i in range(self.M):
            v += residual[i,0]**2
        return math.sqrt(v)

    def GetStateDifferenceLength (self, state_difference):
        v = 0 
        for i in range(self.L):
            v += state_difference[i,0]**2
        return math.sqrt(v)


NameError: name 'UnscentedKalmanFilter' is not defined

In [ ]:
e = Example1UKF()
e.RunAllIterations(0,1000)
e.monitor.MakePlots(5)